<a href="https://colab.research.google.com/github/cundeyu154/PL-Repo/blob/main/%E4%BD%9C%E6%A5%AD%E4%BA%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade google-generativeai pandas gspread googlemaps gradio python-dateutil

In [2]:
import os
import time
import pandas as pd
import numpy as np
import random
import gradio as gr
import asyncio
import googlemaps
from dateutil.tz import gettz
from urllib.parse import quote
SHEET_URL = "https://docs.google.com/spreadsheets/d/1i66JjMWDJCVbGGvQyGfHcPNUxMHke9bjAswfhy-TbqE/edit?gid=0#gid=0"
WORKSHEET_NAME = "模擬資料"

GEMINI_API_KEY = None
MAPS_API_KEY = None
model = None
gmaps = None

try:
    from google.colab import userdata
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    MAPS_API_KEY = userdata.get('Maps_api_key')
    print("Colab Secrets 讀取嘗試成功。")

    if not GEMINI_API_KEY:
        print("警告：'GEMINI_API_KEY' 在 Colab Secrets 中為空或未設定。")
    if not MAPS_API_KEY:
        print("警告：'Maps_api_key' 在 Colab Secrets 中為空或未設定。")

except Exception as e:
    print(f"致命警告：userdata.get() 讀取密鑰失敗或超時。錯誤: {e}")
    print("將嘗試從環境變數讀取。")
    GEMINI_API_KEY = os.environ.get('GEMINI_API_KEY')
    MAPS_API_KEY = os.environ.get('Maps_api_key')

try:
    from google.colab import auth
    import gspread
    from google.auth import default
    import google.generativeai as genai

    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
    print("Google Colab 身份驗證成功。")
    if GEMINI_API_KEY:
        genai.configure(api_key=GEMINI_API_KEY)
        model = genai.GenerativeModel('gemini-2.5-pro')
        print("Gemini API 配置成功。")
    else:
        raise ValueError("GEMINI_API_KEY 為空，無法配置 Gemini 模型。")

    if MAPS_API_KEY:
        gmaps = googlemaps.Client(key=MAPS_API_KEY)
        print("Google Maps Client 配置成功。")

    df_restaurants = None

except Exception as e:
    print(f"\n環境設定失敗，使用模擬資料與功能。錯誤: {e}")
    data = {
        '店名': ['TBT 靜巷咖啡', 'WES 共享辦公', 'COF 頂級精品', 'BAR 深夜酒吧', 'SBU 星光門市'],
        '風格': ['文青/安靜', '辦公/插座', '高品質/約會', '休閒/微醺', '大眾/方便'],
        '預算': ['$$', '$$$', '$$$', '$$$', '$'],
        '距離': ['短', '中', '長', '中', '短'],
        '開放中': ['是', '是', '是', '否', '是'],
        '座標': ['25.0330,121.5454', '25.0430,121.5554', '25.0530,121.5654', '25.0630,121.5754', '25.0350,121.5500']
    }
    df_restaurants = pd.DataFrame(data)
    model = None
    MAPS_API_KEY = "MOCK_MAPS_KEY"
    print("已切換至模擬模式，請檢查 Colab Secrets 中的 API Key 是否正確。")


# 載入 Google Sheet 資料
if 'gc' in locals():
    try:
        gsheets = gc.open_by_url(SHEET_URL)
        sheets = gsheets.worksheet(WORKSHEET_NAME).get_all_values()
        df_restaurants = pd.DataFrame(sheets[1:], columns=sheets[0])
        df_restaurants.columns = ['店名', '風格', '預算', '距離', '開放中', '座標']
        print("Google Sheet 咖啡廳資料載入成功。")
    except Exception as e:
        print(f"載入 Google Sheet 失敗，使用模擬資料。錯誤: {e}")
        if df_restaurants is None:
            df_restaurants = pd.DataFrame(data)

def filter_and_recommend(df, budget_filter, distance_filter, is_open):
    """根據使用者輸入篩選並隨機推薦 3 家咖啡廳"""
    budget_list = budget_filter if budget_filter else df['預算'].unique()
    distance_list = distance_filter if distance_filter else df['距離'].unique()
    open_status = ['是'] if is_open else ['是', '否']

    filtered_df = df[
        df['預算'].isin(budget_list) &
        df['距離'].isin(distance_list) &
        df['開放中'].isin(open_status)
    ].copy()

    if filtered_df.empty:
        return pd.DataFrame(), "沒有找到符合條件的咖啡廳。", "", ""

    n_recommend = min(3, len(filtered_df))
    recommended_df = filtered_df.sample(n=n_recommend, replace=False).reset_index(drop=True)

    recommend_list = "\n".join([
        f"{i+1}. {row['店名']} ({row['預算']} / {row['距離']}) - 風格: {row['風格']}"
        for i, row in recommended_df.iterrows()
    ])

    ai_input = "、".join([
        f"{row['店名']}，預算{row['預算']}，距離{row['距離']}，風格：{row['風格']}"
        for _, row in recommended_df.iterrows()
    ])
    first_cafe_name = recommended_df.iloc[0]['店名']
    map_html = generate_maps_embed(first_cafe_name)

    return recommended_df, recommend_list, ai_input, map_html
def generate_maps_embed(query):
    """生成 Google Maps 嵌入網址的 HTML (使用 gr.HTML 顯示)"""

    full_query = f"台北市 {query}"
    encoded_query = quote(full_query)

    if not MAPS_API_KEY or MAPS_API_KEY == "MOCK_MAPS_KEY":
        html = f"""
        <p>⚠️ **注意：請設置 Google Maps API Key，目前顯示的是通用搜索。**</p>
        <iframe width="100%" height="300" src="https://maps.google.com/maps?q={encoded_query}&t=&z=15&ie=UTF8&iwloc=&output=embed" frameborder="0" style="border:0" allowfullscreen></iframe>
        """
        return html
    embed_url = f"https://www.google.com/maps/embed/v1/search?key={MAPS_API_KEY}&q={encoded_query}"

    html = f"""
    <p>☕ **推薦地點：{query}**</p>
    <iframe width="100%" height="300" frameborder="0" style="border:0"
        src="{embed_url}" allowfullscreen>
    </iframe>
    """
    return html


Colab Secrets 讀取嘗試成功。
Google Colab 身份驗證成功。
Gemini API 配置成功。
Google Maps Client 配置成功。
Google Sheet 咖啡廳資料載入成功。


In [ ]:
async def generate_summary_real(ai_input):
    """呼叫 LLM 產生決策建議摘要 (使用 gemini-2.5-pro)"""
    if not ai_input or model is None:
        await asyncio.sleep(1)
        mock_summary = f"""
🤖 **[模擬 AI 助理]** 今天的充電選項是：{'、'.join([r.split('，')[0].strip() for r in ai_input.split('、')])}。
請檢查您的 API Key 是否有效或配額是否用盡，以啟用真實 AI 推薦。
"""
        return mock_summary.strip()


    system_prompt = (
        "您是一位頂尖的遠距工作與咖啡廳氛圍顧問。您的任務是根據提供的咖啡廳清單，"
        "分析其優缺點（如預算、距離、風格）並總結，幫助使用者在 3 家中快速做出決定。"
        "請以專業且鼓舞人心的口吻，用 **繁體中文** 撰寫一個約 **100 字** 的決策建議摘要。"
        "請避免使用 Markdown 標題和編號清單。"
    )

    user_query = f"這是三家推薦的咖啡廳清單：{ai_input}。請綜合考量其預算、距離和風格，為我提供一份快速決策建議，讓我能選擇最適合充電或趕工的地點。"

    full_prompt = f"### 系統指令 ###\n{system_prompt}\n\n### 用戶查詢 ###\n{user_query}\n\n請根據以上指令與資訊，生成決策建議摘要。"

    try:
        response = await asyncio.to_thread(
            model.generate_content,
            contents=full_prompt,
        )
        return response.text.strip()
    except Exception as e:
        return f"呼叫 Gemini API 失敗，請檢查 API 金鑰或配額：{e}。"


async def wrapper_recommend(budget_filter, distance_filter, is_open):
    """Gradio 主要流程：篩選 -> 推薦 -> 呼叫 AI 摘要 -> 地圖"""
    recommended_df, recommend_list, ai_input, map_html = filter_and_recommend(
        df_restaurants, budget_filter, distance_filter, is_open
    )

    ai_summary = await generate_summary_real(ai_input)

    display_df = recommended_df[['店名', '預算', '距離']].reset_index(drop=True)

    if display_df.empty:
        return pd.DataFrame(), "沒有找到符合條件的咖啡廳。", "（無建議）", generate_maps_embed("台北市")

    return display_df, recommend_list, ai_summary, map_html

# --- Gradio 介面 ---

with gr.Blocks(title="科技人充電咖啡廳決策器") as demo:
    gr.Markdown(
        """
        # ☕ 科技人下班後的「充電」咖啡廳決策器
        ### 讓 AI 顧問從台北市的咖啡廳清單中，為你推薦最適合趕工或放鬆的黃金地點！
        """
    )

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("## 條件篩選 (你的需求)")
            budget_select = gr.CheckboxGroup(
                ['$', '$$', '$$$'],
                label="預算範圍 (可複選)",
                value=['$', '$$'],
            )
            distance_select = gr.CheckboxGroup(
                ['短', '中', '長'],
                label="距離過濾 (可複選)",
                value=['短', '中'],
            )
            open_toggle = gr.Checkbox(
                label="只看目前開放",
                value=True
            )
            recommend_btn = gr.Button("🚀 開始推薦 / 重新抽取")

        with gr.Column(scale=2):
            gr.Markdown("## 推薦清單 (隨機 3 家)")
            output_dataframe = gr.DataFrame(
                headers=["店名", "預算", "距離"],
                datatype=["str", "str", "str"],
                label="隨機推薦結果 (簡表)"
            )
            output_list = gr.Textbox(label="推薦細節 (含風格)", lines=3)

    gr.Markdown("## 🤖 AI 決策顧問報告")
    output_summary = gr.Textbox(
        label="AI 決策顧問報告",
        interactive=False,
        lines=5
    )
    gr.Markdown("## 📍 推薦地點地圖 (第一名地點)")
    output_map = gr.HTML(
        label="地圖顯示",
        visible=True,
        value=generate_maps_embed("台北市")
    )
    recommend_btn.click(
        fn=wrapper_recommend,
        inputs=[budget_select, distance_select, open_toggle],
        outputs=[output_dataframe, output_list, output_summary, output_map]
    )


# 運行 Gradio 應用
if __name__ == "__main__":
    print("Gradio 應用正在啟動...")
    demo.launch(inbrowser=True, debug=True)

Gradio 應用正在啟動...
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://3d668d9857a3582ad2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
